In [ ]:
def myintersect(a, b):
    if (len(a) == len(b)):
        result = []
        count = 0
        for i in range(len(a)):
            if (a[i] == b[i]):
                result.append(a[i])
                count += 1
            else:
                result.append(MyErrorWord)
        return result, count 
    else:
        return 'Error(not corresponds dimentions)', 0
def issub(a, b):
    if (len(a) == len(b)):
        result = []
        count = 0
        for i in range(len(a)):
            if (a[i]!= MyErrorWord):
                if (a[i] != b[i]):
                    return False
        return True
    else:
        return 'Error(not corresponds dimentions)'